<a href="https://colab.research.google.com/github/njucs/notebook/blob/master/LMM_Design_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLM Design Patterns
- 模型响应太慢
    - 分层缓存 - Layered Caching
- 需要与已有系统整合
    - 工具利用 - Tool Use
    - 规则融合 - Blending Rules Based & Generative
- 问题过于复杂
    - 多步规划 - Planning
    - 多重解析 - Discriminate→Re-Compose→ReSolve→Re-Decompose (Dr3)
    - 记忆认知- Memory Cognition For LLM's
- 需要集合多个视角解决问题
    - 专家集成 - Multiplexing AI Agents For A Panel Of Experts
    - 群体智能 - Swarm Of Generative AI Agents
- 多功能性和适应性不足
    - 多任务微调 - Fine-Tuning LLM's For Multiple Tasks
    - 模块化组合 - Modular Monolith LLM Approach With Composability
- 输出质量和用户体验不可控
    - 知识库驱动 - Utilizing Knowledge Graphs with LLM's
    - 双模评估 - Red & Blue Team Dual-Model Evaluation

### 分层缓存 Layered Caching
![图片描述](https://drive.google.com/uc?id=1_Xk0IIZvsNA7_Si-5qMyzvye9LxumQOd)

In [ ]:
# -*- coding: utf-8 -*-
"""
# LLM 查询处理流程演示

本 Jupyter Notebook 演示了一个使用大型语言模型（LLM）进行查询处理的系统流程。
该流程对应了图片中的结构，模拟了初始查询、缓存、后续查询、新查询、大型LLM、
小型LLM微调以及最终输出等环节。
"""

import random
from typing import Dict, Callable

# --- 1. 模拟查询 ---
def generate_query(query_type: str):
    """
    生成模拟查询
    Args:
        query_type: 查询类型，如 "initial", "follow-up", "new"
    Returns:
        查询文本
    """
    if query_type == "initial":
        return f"Initial Query {random.randint(1, 100)}"
    elif query_type == "follow-up":
        return f"Follow-up Query {random.randint(1, 100)}"
    elif query_type == "new":
        return f"New Query {random.randint(1, 100)}"
    else:
        return "Invalid Query Type"

# --- 2. 模拟大型LLM ---
def mock_large_llm(query: str):
    """
    模拟大型LLM的处理
    Args:
        query: 查询文本
    Returns:
        LLM 的模拟响应
    """
    return f"Large LLM Response for: {query}"

# --- 3. 模拟小型LLM ---
class SmallLLM:
    def __init__(self, name: str, training_data: dict = None):
        """
        初始化小型LLM
        Args:
            name: 小型LLM名称
            training_data: 用于微调的训练数据，格式为 {query: response}
        """
        self.name = name
        self.training_data = training_data if training_data else {} # 如果没有传递，则初始化为空字典

    def process_query(self, query: str):
        """
        处理查询
        Args:
            query: 查询文本
        Returns:
            小型LLM的模拟响应
        """
        # 模拟微调效果：如果查询在训练数据中，直接返回训练数据中的响应，否则返回默认响应
        if query in self.training_data:
             return f"Fine-tuned Small LLM Response from training data for: {query}, result: {self.training_data[query]}"
        else:
            return f"Fine-tuned Small LLM Response for: {query}"


    def fine_tune(self, query, response):
        """
         使用新的数据对小型LLM进行微调
        Args:
            query: 查询文本
            response: 对应的响应文本
        """
        self.training_data[query] = response
        print(f"Small LLM '{self.name}' has been fine-tuned with query: '{query}' and response: '{response}' ")

# --- 4. 模拟缓存 ---
class QueryCache:
    def __init__(self):
        """
        初始化查询缓存
        """
        self.cache = {}

    def check_cache(self, query: str):
        """
        检查缓存中是否存在查询
        Args:
            query: 查询文本
        Returns:
            缓存结果，如果存在，否则返回 None
        """
        return self.cache.get(query)

    def add_to_cache(self, query: str, response: str):
        """
        将查询和响应添加到缓存
        Args:
             query: 查询文本
             response: LLM 的响应文本
        """
        self.cache[query] = response
        print(f"Query '{query}' added to the cache.")

# --- 5. 系统流程函数 ---
def process_query_system(query_type: str, large_llm_func: Callable = mock_large_llm, cache: QueryCache = None, small_llm: SmallLLM = None):
    """
    处理不同类型的查询，并输出结果
    Args:
        query_type: 查询类型，如 "initial", "follow-up", "new"
        large_llm_func: 用于处理查询的大型LLM函数
        cache: 用于查询缓存
        small_llm:  用于处理新查询的小型LLM实例
    Returns:
        查询处理结果
    """
    query = generate_query(query_type)
    print(f"\nReceived {query_type} query: '{query}'")
    if query_type == "initial":
        llm_response = large_llm_func(query)
        if cache:
            cache.add_to_cache(query, llm_response)
            return llm_response
        else:
            return llm_response

    elif query_type == "follow-up":
        if cache:
            cached_response = cache.check_cache(query)
            if cached_response:
                print("  Cache hit.")
                return cached_response
            else:
                print("  Cache miss, using Large LLM")
                llm_response = large_llm_func(query)
                if cache:
                    cache.add_to_cache(query, llm_response)
                return llm_response
        else:
            llm_response = large_llm_func(query)
            return llm_response

    elif query_type == "new":
        if small_llm:
            print('  Using Small LLM')
            small_llm_response = small_llm.process_query(query)
            return small_llm_response
        else:
            print('  No Small LLM available, using large LLM')
            llm_response = large_llm_func(query)
            return llm_response


    else:
        return "Invalid query type"

# --- 6. 主函数 ---
if __name__ == '__main__':
    print("# Start Simulation #\n")
    # 初始化缓存和小型LLM
    cache = QueryCache()
    small_llm = SmallLLM('Specialized-LLM')

    # 初始查询
    initial_output = process_query_system("initial", cache=cache)
    print("  Output:", initial_output)

    # 后续查询，第一次缓存命中
    follow_up_output_1 = process_query_system("follow-up", cache=cache)
    print("  Output:", follow_up_output_1)

    # 后续查询，第二次，不在缓存中
    follow_up_output_2 = process_query_system("follow-up", cache=cache)
    print("  Output:", follow_up_output_2)

    # 新查询， 使用未微调的小型LLM
    new_output_1 = process_query_system("new", small_llm=small_llm)
    print("  Output:", new_output_1)

    # 模拟微调
    small_llm.fine_tune(generate_query('new'), 'Fine-tuned Response')

    # 新查询，使用微调后的小型LLM
    new_output_2 = process_query_system("new", small_llm=small_llm)
    print("  Output:", new_output_2)

# Start Simulation #


Received initial query: 'Initial Query 23'
Query 'Initial Query 23' added to the cache.
  Output: Large LLM Response for: Initial Query 23

Received follow-up query: 'Follow-up Query 25'
  Cache miss, using Large LLM
Query 'Follow-up Query 25' added to the cache.
  Output: Large LLM Response for: Follow-up Query 25

Received follow-up query: 'Follow-up Query 84'
  Cache miss, using Large LLM
Query 'Follow-up Query 84' added to the cache.
  Output: Large LLM Response for: Follow-up Query 84

Received new query: 'New Query 66'
  Using Small LLM
  Output: Fine-tuned Small LLM Response for: New Query 66
Small LLM 'Specialized-LLM' has been fine-tuned with query: 'New Query 47' and response: 'Fine-tuned Response' 

Received new query: 'New Query 19'
  Using Small LLM
  Output: Fine-tuned Small LLM Response for: New Query 19


In [1]:
import time
import datetime

# In-memory cache for demonstration purposes
cache = {}

def call_large_llm(query_text):
    """Placeholder to interact with a large LLM (e.g., GPT/Gemini)."""
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Calling large LLM with query: {query_text}")
    # Simulate some processing time
    time.sleep(1)  # Uncomment to simulate time taken
    return f"large_llm_response for '{query_text}'"

def call_fine_tuned_llm(query_text):
    """Placeholder to interact with a specific/fine-tuned LLM."""
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Calling fine-tuned LLM with query: {query_text}")
    time.sleep(0.5)
    return f"fine_tuned_llm_response for '{query_text}'"

def cache_result(query_text, result):
    """Stores the query and the result in the cache."""
    cache[query_text] = result
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Result cached for query: {query_text}")

def get_cached_result(query_text):
    """Fetches a result from the cache, returns None if not found."""
    return cache.get(query_text)

def output(result):
    """Placeholder to output the result."""
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Output: {result}")

def process_query(query_type, query_text):
    """Main function to process queries based on their type."""
    if query_type == "initial":
        result = call_large_llm(query_text)
        cache_result(query_text, result)
        output(result)
    elif query_type == "follow-up":
        cached_result = get_cached_result(query_text)
        if cached_result:
            print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Cache hit for query: {query_text}")
            output(cached_result)
        else:
            result = call_large_llm(query_text)
            cache_result(query_text, result)
            output(result)
    elif query_type == "new":
        result = call_fine_tuned_llm(query_text)
        output(result)
    else:
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Invalid query type: {query_type}")

# Example Usage:
process_query("initial", "What is the capital of France?")
process_query("follow-up", "What is the capital of France?") # Should hit the cache
process_query("follow-up", "Tell me more about France")
process_query("new", "Translate hello to French.")
process_query("invalid", "invalid query.")

12:19:18: Calling large LLM with query: What is the capital of France?
12:19:19: Result cached for query: What is the capital of France?
12:19:19: Output: large_llm_response for 'What is the capital of France?'
12:19:19: Cache hit for query: What is the capital of France?
12:19:19: Output: large_llm_response for 'What is the capital of France?'
12:19:19: Calling large LLM with query: Tell me more about France
12:19:20: Result cached for query: Tell me more about France
12:19:20: Output: large_llm_response for 'Tell me more about France'
12:19:20: Calling fine-tuned LLM with query: Translate hello to French.
12:19:21: Output: fine_tuned_llm_response for 'Translate hello to French.'
12:19:21: Invalid query type: invalid


### 工具利用 Tool Use
![图片描述](https://drive.google.com/uc?id=1UFLljvksBOc_MhxkYr25vxABvPKU2WN4)

### 规则融合 Blending Rules Based & Generative
![图片描述](https://drive.google.com/uc?id=1tUuD2a3EwbQKEuFWsD0_Y4XLB4q9pFxp)

In [3]:
# 导入必要的库
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# 定义规则引擎函数
def rule_engine(input_text):
    # 这里可以添加具体的规则逻辑
    if 'example' in input_text:
        return "This is an example."
    else:
        return None

# 定义生成式模型函数
def generative_model(input_text):
    model_name = "t5-small"  # 使用T5模型作为示例
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
    output = generator(input_text, max_length=50, num_return_sequences=1)
    return output[0]['generated_text']

# 定义规则验证函数
def rule_validation(input_text, generated_text):
    # 这里可以添加具体的规则验证逻辑
    if 'example' in generated_text:
        return True
    else:
        return False

# 定义人工校对函数（模拟）
def manual_correction(generated_text):
    # 模拟人工校对过程
    return "Corrected text."

# 主函数
def main():
    # 获取用户输入
    user_input = input("请输入文本: ")

    # 规则引擎处理
    rule_output = rule_engine(user_input)

    if rule_output:
        print("规则引擎输出:", rule_output)
    else:
        # 生成式模型生成内容
        generated_text = generative_model(user_input)
        print("生成式模型输出:", generated_text)

        # 规则验证
        if rule_validation(user_input, generated_text):
            print("验证通过，最终输出:", generated_text)
        else:
            # 人工校对
            corrected_text = manual_correction(generated_text)
            print("人工校对后的输出:", corrected_text)

if __name__ == "__main__":
    main()

请输入文本: example
规则引擎输出: This is an example.


In [2]:
import datetime
import time

# Placeholder for LLM generated rules
llm_rules = []

# Placeholder for manual rules
manual_rules = []

# Placeholder for logs and feedback
logs = []


def apply_rules(input_text):
  """Applies rules to the input and generates content."""
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Applying rules to input: {input_text}")
  # Combining both manual and LLM generated rules
  combined_rules = manual_rules + llm_rules
  if combined_rules:
    rule_applied = False
    for rule in combined_rules:
      if rule in input_text:
        rule_applied = True
        generated_content = f"Generated content using rule: '{rule}' for input: '{input_text}'"
        break # Only apply the first rule it matches
    if not rule_applied:
      generated_content = f"No matching rules for input: '{input_text}'"
  else:
    generated_content = f"No rules available for input: '{input_text}'"
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Content generated: {generated_content}")
  return generated_content

def generate_query_model(generated_content):
  """Generates a model for processing queries based on the generated content."""
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Generating query processing model from content: {generated_content}")
  time.sleep(0.5)
  model =  f"Model for processing queries based on generated content '{generated_content}'"
  return model

def validate_model(model):
  """Validates the query processing model."""
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Validating query model: {model}")
  time.sleep(0.5)
  # Simulate validation (e.g. based on metrics and thresholds)
  is_valid =  'valid' in model.lower()
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Model validation result: {'valid' if is_valid else 'invalid'}")
  return is_valid

def log_and_feedback(validation_result, input_text, model):
  """Logs the interaction and provides feedback."""
  log_entry = {
        "timestamp": datetime.datetime.now(),
        "input_text": input_text,
        "validation_result": validation_result,
        "model":model
    }
  logs.append(log_entry)
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Logged interaction: {log_entry}")
  if not validation_result:
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Feedback: Model is invalid. Consider refining rules based on the logs.")

def adjust_rules(input_text):
  """Adjusts the rules based on feedback, simulated manual adjustment."""
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Adjusting rules based on previous input: {input_text}")
  # Simulate manual adjustment based on input, for example by adding a new rule.
  manual_rules.append(f"manual rule based on input: '{input_text}'")
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Manual rules after adjusting:{manual_rules}")

def generate_llm_rule():
  """Placeholder to simulate LLM generating a rule."""
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Simulating LLM generating rules.")
  new_rule = f"llm generated rule at {datetime.datetime.now().strftime('%H:%M:%S')}"
  llm_rules.append(new_rule)
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: LLM Rules after generated rule:{llm_rules}")

def process_input(input_text):
  """Main function to process input using rules and query model."""
  generated_content = apply_rules(input_text)
  model = generate_query_model(generated_content)
  validation_result = validate_model(model)
  log_and_feedback(validation_result, input_text, model)

  if not validation_result:
    adjust_rules(input_text)
    # Process input again with the adjusted rules
    # recursive call will cause infinite loops in this case, it is just an example.
    # process_input(input_text)
  else:
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: Final output for input {input_text}")

# Example Usage:
generate_llm_rule()
process_input("input with a rule1")
process_input("another input with rule2")
process_input("input without rule")
process_input("input with a valid model")

# Log inspection
print(f"\nFinal Logs: {logs}")

00:55:24: Simulating LLM generating rules.
00:55:24: LLM Rules after generated rule:['llm generated rule at 00:55:24']
00:55:24: Applying rules to input: input with a rule1
00:55:24: Content generated: No matching rules for input: 'input with a rule1'
00:55:24: Generating query processing model from content: No matching rules for input: 'input with a rule1'
00:55:25: Validating query model: Model for processing queries based on generated content 'No matching rules for input: 'input with a rule1''
00:55:25: Model validation result: invalid
00:55:25: Logged interaction: {'timestamp': datetime.datetime(2025, 1, 6, 0, 55, 25, 784489), 'input_text': 'input with a rule1', 'validation_result': False, 'model': "Model for processing queries based on generated content 'No matching rules for input: 'input with a rule1''"}
00:55:25: Feedback: Model is invalid. Consider refining rules based on the logs.
00:55:25: Adjusting rules based on previous input: input with a rule1
00:55:25: Manual rules aft

### 多步规划 Planning
![图片描述](https://drive.google.com/uc?id=12y2AVBs81nfgPYwcYeJY-Sl8IHN7Hxrn)


In [2]:
# 确保你已经安装了必要的库，如 transformers 和 torch。如果你还没有安装它们，可以通过以下命令安装
!pip install transformers torch

In [3]:
# 导入所需的库
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# 定义任务规划函数
def plan_task(input_text):
    # 简单的任务规划，这里假设所有输入都需要情感分析
    print("任务规划: 需要进行情感分析.")
    return "sentiment_analysis"

# 定义模型选择函数
def select_model(task_type):
    if task_type == "sentiment_analysis":
        model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
        classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
        print(f"选择了模型: {model_name}")
        return classifier
    else:
        raise ValueError("不支持的任务类型")

# 定义任务执行函数
def execute_task(classifier, input_text):
    result = classifier(input_text)
    print(f"任务执行结果: {result}")
    return result

# 定义响应生成函数
def generate_response(result):
    sentiment = result[0]['label']
    score = result[0]['score']

    if 'positive' in sentiment.lower():
        response = f"这段文本看起来是一个积极的评价，置信度为 {score:.2f}."
    elif 'negative' in sentiment.lower():
        response = f"这段文本看起来是一个消极的评价，置信度为 {score:.2f}."
    else:
        response = "无法明确判断这段文本的情感倾向."

    print(f"最终响应: {response}")
    return response

# 主函数，整合以上步骤
def main(input_text):
    try:
        # 任务规划
        task_type = plan_task(input_text)

        # 模型选择
        classifier = select_model(task_type)

        # 任务执行
        result = execute_task(classifier, input_text)

        # 响应生成
        final_response = generate_response(result)

        return final_response

    except Exception as e:
        print(f"发生错误: {e}")

# 测试主函数
if __name__ == "__main__":
    user_input = "我非常喜欢这个产品！它真的改变了我的生活。"  # 示例输入
    main(user_input)

任务规划: 需要进行情感分析.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Device set to use cpu


选择了模型: nlptown/bert-base-multilingual-uncased-sentiment
任务执行结果: [{'label': '5 stars', 'score': 0.656829833984375}]
最终响应: 无法明确判断这段文本的情感倾向.


In [1]:
import time
import datetime
import random

# 模拟外部工具或模型，例如天气API，计算器，翻译器等
class ExternalTool:
    def __init__(self, name, description):
        self.name = name
        self.description = description

    def execute(self, input_data):
         """ 模拟执行工具，返回结果"""
         print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 执行工具 '{self.name}'，输入: '{input_data}'")
         time.sleep(random.uniform(0.5, 1.5))  # 模拟执行时间
         if self.name == "天气API":
             return f"当前天气是晴朗，温度25度"
         elif self.name == "计算器":
            try:
                return str(eval(input_data))
            except:
                return "计算错误"
         elif self.name == "翻译器":
             return f"翻译后的结果: '你好' is 'hello'"
         else:
            return f"工具 '{self.name}' 无法处理输入 '{input_data}'"


#  定义可用的工具
available_tools = [
    ExternalTool("天气API", "查询当前天气情况"),
    ExternalTool("计算器", "执行数学计算"),
    ExternalTool("翻译器", "进行文本翻译"),
]

# 任务规划器
def task_planner(user_request):
    """
    任务规划器，将用户请求分解为子任务。
    这里使用简单的规则来演示，在实际应用中，可能需要更复杂的逻辑。
    """
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 规划任务，用户请求：'{user_request}'")
    tasks = []
    if "天气" in user_request:
       tasks.append({"tool": "天气API", "input": None})
    if "计算" in user_request or "+" in user_request or "-" in user_request or "*" in user_request or "/" in user_request:
      tasks.append({"tool": "计算器", "input": user_request.split("计算")[-1].strip() if "计算" in user_request else user_request})
    if "翻译" in user_request:
      tasks.append({"tool": "翻译器", "input":"你好"}) # 这里翻译器模拟翻译一个固定的词
    if not tasks:
        return None # 如果没有匹配的任务，返回None

    return tasks


# 模型选择器，根据任务选择合适的工具
def model_selector(task):
    """
     根据任务选择合适的工具。
    """
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 选择模型，任务: {task}")
    for tool in available_tools:
        if tool.name == task['tool']:
            return tool
    return None


# 任务执行器
def task_executor(tool, input_data):
    """
    执行选择的工具，并返回结果。
    """
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 执行任务，使用工具 '{tool.name}'，输入：'{input_data}'")
    if tool:
      return tool.execute(input_data)
    else:
        return "没有合适的工具执行该任务"


# 响应生成器
def response_generator(results, user_request):
    """
    根据任务结果生成响应。
    """
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 生成响应，结果: {results}，用户请求：'{user_request}'")
    response = ""
    if len(results) == 0:
       return "无法处理该请求，请尝试其他请求。"
    if len(results) == 1:
        response = f"根据您的请求： '{user_request}'，结果是：{results[0]}"
    else:
        response = f"根据您的请求：'{user_request}'， 结果是："
        for i, result in enumerate(results):
            response += f"任务 {i+1}: {result}; "

    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 生成的响应：{response}")
    return response

# 主流程
def process_user_request(user_request):
    """
    处理用户请求的主流程。
    """
    tasks = task_planner(user_request)

    if not tasks:
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 无法识别用户请求，请提供更详细的指令")
      return "无法识别用户请求，请提供更详细的指令"

    results = []
    for task in tasks:
      tool = model_selector(task)
      result = task_executor(tool, task.get("input",None))
      results.append(result)
    response = response_generator(results, user_request)
    return response


# 示例用户请求
user_requests = [
    "今天天气怎么样？",
    "计算 123 + 456",
    "翻译 你好",
    "查询天气并计算 2 * 3",
    "我需要一个蛋糕"
]

for request in user_requests:
    print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')}: 用户请求：{request}")
    response = process_user_request(request)
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 系统响应：{response}")


05:38:34: 用户请求：今天天气怎么样？
05:38:34: 规划任务，用户请求：'今天天气怎么样？'
05:38:34: 选择模型，任务: {'tool': '天气API', 'input': None}
05:38:34: 执行任务，使用工具 '天气API'，输入：'None'
05:38:34: 执行工具 '天气API'，输入: 'None'
05:38:35: 生成响应，结果: ['当前天气是晴朗，温度25度']，用户请求：'今天天气怎么样？'
05:38:35: 生成的响应：根据您的请求： '今天天气怎么样？'，结果是：当前天气是晴朗，温度25度
05:38:35: 系统响应：根据您的请求： '今天天气怎么样？'，结果是：当前天气是晴朗，温度25度

05:38:35: 用户请求：计算 123 + 456
05:38:35: 规划任务，用户请求：'计算 123 + 456'
05:38:35: 选择模型，任务: {'tool': '计算器', 'input': '123 + 456'}
05:38:35: 执行任务，使用工具 '计算器'，输入：'123 + 456'
05:38:35: 执行工具 '计算器'，输入: '123 + 456'
05:38:36: 生成响应，结果: ['579']，用户请求：'计算 123 + 456'
05:38:36: 生成的响应：根据您的请求： '计算 123 + 456'，结果是：579
05:38:36: 系统响应：根据您的请求： '计算 123 + 456'，结果是：579

05:38:36: 用户请求：翻译 你好
05:38:36: 规划任务，用户请求：'翻译 你好'
05:38:36: 选择模型，任务: {'tool': '翻译器', 'input': '你好'}
05:38:36: 执行任务，使用工具 '翻译器'，输入：'你好'
05:38:36: 执行工具 '翻译器'，输入: '你好'
05:38:37: 生成响应，结果: ["翻译后的结果: '你好' is 'hello'"]，用户请求：'翻译 你好'
05:38:37: 生成的响应：根据您的请求： '翻译 你好'，结果是：翻译后的结果: '你好' is 'hello'
05:38:37: 系统响应：根据您的请求： '翻译 你好'，结果是：翻译后的

### 多重解析 - Discriminate→Re-Compose→ReSolve→Re-Decompose (Dr3)
![图片描述](https://drive.google.com/uc?id=17N5FeE8SIUSpjO9n7cM-CfjjeLtmJ-t4)

In [ ]:
# https://github.com/Gy915/Dr3.git

### 记忆认知 Memory Cognition For LLM’s
![图片描述](https://drive.google.com/uc?id=1aQRLlQER_MckkuKbwpwZdzQ-7Pum_VfD)


In [4]:
# 导入必要的库
import pandas as pd
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# 定义短期记忆库和长期记忆库
short_term_memory = []
long_term_memory = []

# 定义大语言模型函数
def llm(input_text):
    model_name = "t5-small"  # 使用T5模型作为示例
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
    output = generator(input_text, max_length=50, num_return_sequences=1)
    return output[0]['generated_text']

# 定义总结器函数
def summarizer(output_text):
    summary_model_name = "facebook/bart-large-cnn"
    summary_tokenizer = AutoTokenizer.from_pretrained(summary_model_name)
    summary_model = AutoModelForSeq2SeqLM.from_pretrained(summary_model_name)
    summary_generator = pipeline('summarization', model=summary_model, tokenizer=summary_tokenizer)
    summary = summary_generator(output_text, max_length=50, num_return_sequences=1)
    return summary[0]['summary_text']

# 定义短期记忆库更新函数
def update_short_term_memory(summary):
    short_term_memory.append(summary)

# 定义长期记忆库更新函数
def update_long_term_memory(summary):
    long_term_memory.append(summary)

# 定义记忆评估与衰退函数
def memory_assessment_and_decay():
    # 这里可以添加具体的记忆评估逻辑
    if len(short_term_memory) > 5:  # 假设短期记忆库最多存储5条记录
        short_term_memory.pop(0)

# 主函数
def main():
    # 获取用户输入
    user_input = input("请输入文本: ")

    # 大语言模型处理
    output_text = llm(user_input)
    print("大语言模型输出:", output_text)

    # 总结器处理
    summary = summarizer(output_text)
    print("总结器输出:", summary)

    # 更新短期记忆库
    update_short_term_memory(summary)

    # 更新长期记忆库
    update_long_term_memory(summary)

    # 记忆评估与衰退
    memory_assessment_and_decay()

    # 打印短期记忆库和长期记忆库
    print("短期记忆库:", short_term_memory)
    print("长期记忆库:", long_term_memory)

if __name__ == "__main__":
    main()

请输入文本: test


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Device set to use cpu
Your min_length=56 must be inferior than your max_length=50.
Your max_length is set to 50, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1427: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


总结器输出: The U.S. military is testing a new system that could be used in the future. The test would be used to test the capabilities of soldiers in the field. It would also be used as a test of the military's ability
短期记忆库: ["The U.S. military is testing a new system that could be used in the future. The test would be used to test the capabilities of soldiers in the field. It would also be used as a test of the military's ability"]
长期记忆库: ["The U.S. military is testing a new system that could be used in the future. The test would be used to test the capabilities of soldiers in the field. It would also be used as a test of the military's ability"]


In [5]:
import time
import datetime
import random

# 模拟大语言模型
class LargeLanguageModel:
    def __init__(self, name):
        self.name = name

    def generate_response(self, input_text, short_term_memory=None):
         """模拟大语言模型生成回复，包含对短期记忆的使用"""
         print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型 '{self.name}' 处理输入：'{input_text}'")
         time.sleep(random.uniform(0.5, 1.5))
         if short_term_memory:
            print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型使用短期记忆: '{short_term_memory}'")
            return f"根据输入 '{input_text}'和短期记忆 '{short_term_memory}'， 大语言模型回复：处理后的结果"

         return f"根据输入 '{input_text}'，大语言模型回复：处理后的结果"

    def generate_query_for_memory(self, input_text):
        """模拟生成用于查询短期记忆的查询语句"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型生成查询语句用于查询短期记忆：'{input_text}'")
        return f"查询：与'{input_text}'相关的信息"

    def generate_information_for_memory(self,input_text):
        """模拟大语言模型生成可以存储到短期记忆中的信息"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型生成可以存储到短期记忆的信息：'{input_text}'")
        return f"与输入 '{input_text}' 相关的信息"

# 短期记忆库
class ShortTermMemory:
    def __init__(self):
        self.memory = []

    def store(self, information):
        """存储信息"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 短期记忆库存储信息：'{information}'")
        self.memory.append(information)

    def query(self, query_text):
        """查询信息"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 短期记忆库查询信息：'{query_text}'")
        if self.memory:
            return random.choice(self.memory)  # 返回一个随机匹配的记忆片段
        return None

    def update(self, information):
       """更新信息"""
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 短期记忆库更新信息，新的信息：'{information}'")
       if self.memory:
          self.memory[-1] = information
       else:
         self.memory.append(information)

    def clear(self):
        """清理记忆"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 短期记忆库清理记忆")
        self.memory = []

# 长期记忆库（使用字典模拟）
class LongTermMemory:
    def __init__(self):
        self.memory = {}  # 使用字典，键为索引，值为信息

    def store(self, information):
      """存储信息，简单使用时间戳作为索引"""
      index = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
      self.memory[index] = information
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 长期记忆库存储信息，索引为 '{index}'， 内容为：'{information}'")

    def retrieve(self, query):
        """根据查询语句，检索长期记忆"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 长期记忆库检索信息：'{query}'")
        if self.memory:
          for key, value in self.memory.items():
            if query in value: # 使用简单匹配，实际可以使用更复杂的索引和检索方式
                return value
          return None  # 如果没有匹配的结果
        else:
            return None


    def index_and_query(self, query):
       """ 模拟索引和查询长期记忆，返回检索结果 """
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 长期记忆库索引并检索信息：'{query}'")
       result = self.retrieve(query)
       return result


# 总结器
class Summarizer:
    def __init__(self, name):
        self.name = name

    def summarize(self, output_text):
        """总结输出，并准备存储到长期记忆库中的信息"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 总结器 '{self.name}' 总结输出：'{output_text}'")
        time.sleep(random.uniform(0.5, 1.0))
        return f"总结后的信息：'{output_text}'"

    def retrieve_known_information(self,output_text):
        """ 模拟获取已知信息 """
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 总结器 '{self.name}' 尝试获取已知信息")
        time.sleep(random.uniform(0.5, 1.0))
        if "历史" in output_text: # 模拟已知一些历史信息
            return f"已知的历史信息： '这是一段历史'"
        else:
            return None

# 记忆评估器
class MemoryEvaluator:
    def __init__(self, name):
         self.name = name

    def evaluate(self, short_term_memory_info):
       """评估短期记忆的价值，并决定是否保留"""
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 记忆评估器 '{self.name}' 评估短期记忆信息：'{short_term_memory_info}'")
       time.sleep(random.uniform(0.5, 1.0))
       # 模拟评估逻辑，随机决定是否保留
       if random.random() > 0.3:  # 70%的概率保留
            print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 记忆评估器 '{self.name}'决定保留短期记忆信息。")
            return True
       else:
            print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 记忆评估器 '{self.name}'决定遗忘短期记忆信息。")
            return False



#  初始化组件
llm = LargeLanguageModel("大语言模型")
short_term_memory = ShortTermMemory()
long_term_memory = LongTermMemory()
summarizer = Summarizer("总结器")
memory_evaluator = MemoryEvaluator("记忆评估器")

# 主流程
def process_input(input_text):
    """处理输入的整体流程"""
    print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')}: 接收到输入: '{input_text}'")

    # 1. 使用大语言模型生成回复，尝试利用短期记忆
    memory_query = llm.generate_query_for_memory(input_text)
    short_term_memory_info = short_term_memory.query(memory_query)

    llm_output = llm.generate_response(input_text, short_term_memory_info)

    # 2. 总结输出，并准备存入长期记忆的信息，以及获取已知的长期记忆信息
    summary_text = summarizer.summarize(llm_output)
    known_information = summarizer.retrieve_known_information(llm_output)

    if known_information: # 如果有已知的长期记忆信息，输出
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 从长期记忆库获取已知信息: '{known_information}'")
    # 3.  存储新的总结信息到长期记忆库
    long_term_memory.store(summary_text)

    # 4.  记忆评估和更新
    short_term_memory_information = llm.generate_information_for_memory(input_text)
    retain_memory = memory_evaluator.evaluate(short_term_memory_information)

    if retain_memory:
       short_term_memory.update(short_term_memory_information)

    # 5. 输出
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 系统输出：{llm_output}")
    return llm_output

# 示例输入
inputs = [
    "今天天气怎么样？",
    "我记得上次你说过今天会下雨",
    "总结一下今天发生的事情",
    "我想了解更多关于历史的信息",
    "更新今天的计划",
    "关于今天，就先这样吧"
]

for input_text in inputs:
    process_input(input_text)
    # 模拟长期记忆库的查询，在每次处理输入之后
    long_term_query = llm.generate_query_for_memory(input_text)
    long_term_result = long_term_memory.index_and_query(long_term_query)
    if long_term_result:
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 长期记忆库的查询结果：'{long_term_result}'\n")
    else:
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 长期记忆库没有查询到相关结果\n")
    time.sleep(1) # 加入时间间隔


06:02:30: 接收到输入: '今天天气怎么样？'
06:02:30: 大语言模型生成查询语句用于查询短期记忆：'今天天气怎么样？'
06:02:30: 短期记忆库查询信息：'查询：与'今天天气怎么样？'相关的信息'
06:02:30: 大语言模型 '大语言模型' 处理输入：'今天天气怎么样？'
06:02:31: 总结器 '总结器' 总结输出：'根据输入 '今天天气怎么样？'，大语言模型回复：处理后的结果'
06:02:32: 总结器 '总结器' 尝试获取已知信息
06:02:33: 长期记忆库存储信息，索引为 '20250106060233011126'， 内容为：'总结后的信息：'根据输入 '今天天气怎么样？'，大语言模型回复：处理后的结果''
06:02:33: 大语言模型生成可以存储到短期记忆的信息：'今天天气怎么样？'
06:02:33: 记忆评估器 '记忆评估器' 评估短期记忆信息：'与输入 '今天天气怎么样？' 相关的信息'
06:02:33: 记忆评估器 '记忆评估器'决定保留短期记忆信息。
06:02:33: 短期记忆库更新信息，新的信息：'与输入 '今天天气怎么样？' 相关的信息'
06:02:33: 系统输出：根据输入 '今天天气怎么样？'，大语言模型回复：处理后的结果
06:02:33: 大语言模型生成查询语句用于查询短期记忆：'今天天气怎么样？'
06:02:33: 长期记忆库索引并检索信息：'查询：与'今天天气怎么样？'相关的信息'
06:02:33: 长期记忆库检索信息：'查询：与'今天天气怎么样？'相关的信息'
06:02:33: 长期记忆库没有查询到相关结果


06:02:34: 接收到输入: '我记得上次你说过今天会下雨'
06:02:34: 大语言模型生成查询语句用于查询短期记忆：'我记得上次你说过今天会下雨'
06:02:34: 短期记忆库查询信息：'查询：与'我记得上次你说过今天会下雨'相关的信息'
06:02:34: 大语言模型 '大语言模型' 处理输入：'我记得上次你说过今天会下雨'
06:02:35: 大语言模型使用短期记忆: '与输入 '今天天气怎么样？' 相关的信息'
06:02:35: 总结器 '总结器' 总结输出：'根据输入 '我记得上次你说过今天会下雨'和短期记忆 '与输入 '今天天气怎么样？' 相

### 专家集成 Multiplexing AI Agents For A Panel Of Experts

![图片描述](https://drive.google.com/uc?id=1UGAWW6her0p4DCdj9ScGZnzIHDUDi5g2)

In [6]:
# 导入必要的库
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd

# 定义通用大语言模型函数
def general_llm(input_text):
    model_name = "t5-small"  # 使用T5模型作为示例
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
    output = generator(input_text, max_length=50, num_return_sequences=1)
    return output[0]['generated_text']

# 定义领域分类器函数
def domain_classifier(input_text):
    # 假设我们有预定义的领域分类器
    domains = ["领域1", "领域2", "领域n"]
    classifier = pipeline("zero-shot-classification")
    result = classifier(input_text, candidate_labels=domains)
    return result['labels'][0]

# 定义小规模领域模型函数
def specific_domain_model(domain, input_text):
    if domain == "领域1":
        model_name = "specific-domain-model-1"
    elif domain == "领域2":
        model_name = "specific-domain-model-2"
    else:
        model_name = "specific-domain-model-n"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
    output = generator(input_text, max_length=50, num_return_sequences=1)
    return output[0]['generated_text']

# 主函数
def main():
    # 获取用户输入
    user_input = input("请输入文本: ")

    # 通用大语言模型处理
    general_output = general_llm(user_input)
    print("通用大语言模型输出:", general_output)

    # 上下文理解（领域分类）
    domain = domain_classifier(user_input)
    print("确定的领域:", domain)

    # 小规模领域模型处理
    specific_output = specific_domain_model(domain, user_input)
    print("小规模领域模型输出:", specific_output)

    # 综合输出
    final_output = f"综合输出: {general_output} | {specific_output}"
    print("综合输出:", final_output)

if __name__ == "__main__":
    main()

请输入文本: 查询一下南京周围主要城市的天气情况


Device set to use cpu
No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


通用大语言模型输出: 


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


确定的领域: 领域1


OSError: specific-domain-model-1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [7]:
import time
import datetime
import random


# 模拟通用大语言模型
class GeneralLanguageModel:
    def __init__(self, name):
        self.name = name

    def understand_context(self, input_text):
        """模拟通用大语言模型理解上下文，并确定输入属于哪个领域"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 通用大语言模型 '{self.name}' 理解上下文：'{input_text}'")
        time.sleep(random.uniform(0.5, 1))
        # 这里使用简单的规则来演示，实际应用中，可以使用更复杂的模型进行领域分类
        if "天气" in input_text:
            return "天气"
        elif "计算" in input_text or "+" in input_text or "-" in input_text or "*" in input_text or "/" in input_text:
            return "计算"
        elif "翻译" in input_text:
             return "翻译"
        else:
            return "通用"  # 如果不属于任何特定领域，返回通用领域

# 模拟特定领域的小模型
class DomainSpecificModel:
    def __init__(self, name, domain):
        self.name = name
        self.domain = domain

    def process_input(self, input_text):
        """模拟特定领域模型处理输入，返回特定领域的输出"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 领域模型 '{self.name}' 处理输入：'{input_text}'")
        time.sleep(random.uniform(0.3, 0.8))
        if self.domain == "天气":
            return f"天气领域模型输出： 今天天气晴朗，温度28度。"
        elif self.domain == "计算":
           try:
             return f"计算领域模型输出： 计算结果是：{eval(input_text)}"
           except:
              return "计算错误"
        elif self.domain == "翻译":
          return f"翻译领域模型输出：翻译结果是：'你好' is 'hello'"

        else:
            return f"通用领域模型输出: 无法处理输入：'{input_text}'"



#  定义可用的领域模型
available_domain_models = {
    "天气": DomainSpecificModel("天气模型", "天气"),
    "计算": DomainSpecificModel("计算模型", "计算"),
    "翻译": DomainSpecificModel("翻译模型", "翻译"),
    "通用": DomainSpecificModel("通用模型", "通用")
}

# 模拟综合输出
class OutputSynthesizer:
  def __init__(self, name):
    self.name = name

  def synthesize(self, outputs):
    """综合不同模型的输出"""
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 综合输出器 '{self.name}' 综合输出：'{outputs}'")
    time.sleep(random.uniform(0.3, 0.7))
    if not outputs:
       return "没有任何输出"
    if len(outputs) == 1:
       return f"综合输出: {outputs[0]}"
    else:
      return f"综合输出:  {', '.join([f'{idx+1}: {output}' for idx, output in enumerate(outputs)])}"

# 初始化组件
general_model = GeneralLanguageModel("通用大语言模型")
output_synthesizer = OutputSynthesizer("综合输出器")

# 主流程
def process_input(input_text):
    """处理输入的整体流程"""
    print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')}: 接收到输入：'{input_text}'")

    # 1. 使用通用大模型理解上下文
    domain = general_model.understand_context(input_text)

    # 2.  使用特定领域的模型处理输入
    if domain in available_domain_models:
        domain_model = available_domain_models[domain]
        output = domain_model.process_input(input_text if domain == "计算" else input_text)
    else:
        output = available_domain_models["通用"].process_input(input_text)


    # 3.  综合输出
    final_output = output_synthesizer.synthesize([output])

    # 4. 输出最终结果
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 系统输出：{final_output}\n")
    return final_output


# 示例输入
inputs = [
    "今天天气怎么样？",
    "计算 123 + 456",
    "翻译 你好",
    "我需要一个蛋糕",
    "今天天气如何？计算 5 * 3",
    "翻译 '再见'",
    "请问现在几点了？"
]

for input_text in inputs:
    process_input(input_text)
    time.sleep(1) # 加入时间间隔


06:06:39: 接收到输入：'今天天气怎么样？'
06:06:39: 通用大语言模型 '通用大语言模型' 理解上下文：'今天天气怎么样？'
06:06:40: 领域模型 '天气模型' 处理输入：'今天天气怎么样？'
06:06:40: 综合输出器 '综合输出器' 综合输出：'['天气领域模型输出： 今天天气晴朗，温度28度。']'
06:06:41: 系统输出：综合输出: 天气领域模型输出： 今天天气晴朗，温度28度。


06:06:42: 接收到输入：'计算 123 + 456'
06:06:42: 通用大语言模型 '通用大语言模型' 理解上下文：'计算 123 + 456'
06:06:42: 领域模型 '计算模型' 处理输入：'计算 123 + 456'
06:06:43: 综合输出器 '综合输出器' 综合输出：'['计算错误']'
06:06:43: 系统输出：综合输出: 计算错误


06:06:44: 接收到输入：'翻译 你好'
06:06:44: 通用大语言模型 '通用大语言模型' 理解上下文：'翻译 你好'
06:06:45: 领域模型 '翻译模型' 处理输入：'翻译 你好'
06:06:46: 综合输出器 '综合输出器' 综合输出：'["翻译领域模型输出：翻译结果是：'你好' is 'hello'"]'
06:06:47: 系统输出：综合输出: 翻译领域模型输出：翻译结果是：'你好' is 'hello'


06:06:48: 接收到输入：'我需要一个蛋糕'
06:06:48: 通用大语言模型 '通用大语言模型' 理解上下文：'我需要一个蛋糕'
06:06:48: 领域模型 '通用模型' 处理输入：'我需要一个蛋糕'
06:06:49: 综合输出器 '综合输出器' 综合输出：'["通用领域模型输出: 无法处理输入：'我需要一个蛋糕'"]'
06:06:49: 系统输出：综合输出: 通用领域模型输出: 无法处理输入：'我需要一个蛋糕'


06:06:50: 接收到输入：'今天天气如何？计算 5 * 3'
06:06:50: 通用大语言模型 '通用大语言模型' 理解上下文：'今天天气如何？计算 5 * 3'
06:06:51: 领域模型 '天气模型' 处理输入：'今天天气如何？计算 5 * 3'
06:06:51: 综合输出器 '综合输出器

### 群体智能 Swarm Of Generative AI Agents

![图片描述](https://drive.google.com/uc?id=1J3OsXz862wmczaRq8t1uMXJE7eJwrevF)


In [9]:
# 导入必要的库
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd

# 定义智能体类
class Agent:
    def __init__(self, name):
        self.name = name
        self.model_name = f"agent-{name}-model"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        self.pipeline = pipeline('text2text-generation', model=self.model, tokenizer=self.tokenizer)

    def process_input(self, input_text):
        # 处理输入并生成输出
        output = self.pipeline(input_text, max_length=50, num_return_sequences=1)
        return output[0]['generated_text']

# 定义共识层集成函数
def consensus_integration(agent_outputs):
    # 对多个智能体的输出进行集成
    consensus_output = " ".join(agent_outputs)
    return consensus_output

# 主函数
def main():
    # 获取用户输入
    user_input = input("请输入文本: ")

    # 智能体路由：选择合适的智能体
    agents = [Agent(f"智能体 {i}") for i in range(1, 6)]  # 创建多个智能体实例
    agent_outputs = []

    # 每个智能体处理输入
    for agent in agents:
        output = agent.process_input(user_input)
        print(f"{agent.name} 的输出: {output}")
        agent_outputs.append(output)

    # 共识层集成
    consensus_output = consensus_integration(agent_outputs)
    print("共识输出:", consensus_output)

    # 最终输出
    final_output = consensus_output
    print("最终输出:", final_output)

    # 反馈机制：性能反馈和调整改进共识集成策略
    feedback = input("请输入反馈: ")
    print("反馈:", feedback)

if __name__ == "__main__":
    main()

请输入文本: 南京天气


OSError: Incorrect path_or_model_id: 'agent-智能体 1-model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [8]:
import time
import datetime
import random

# 模拟智能体
class IntelligentAgent:
    def __init__(self, name, expertise):
        self.name = name
        self.expertise = expertise

    def process_input(self, input_text):
        """模拟智能体处理输入，根据专业领域返回不同的输出"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 智能体 '{self.name}' 处理输入：'{input_text}'")
        time.sleep(random.uniform(0.3, 0.8))
        if self.expertise == "天气":
            return f"智能体 '{self.name}': 天气预报是晴天"
        elif self.expertise == "计算":
            try:
                return f"智能体 '{self.name}': 计算结果：{eval(input_text)}"
            except:
                return f"智能体 '{self.name}': 计算错误"
        elif self.expertise == "翻译":
             return f"智能体 '{self.name}': 翻译结果：'你好' is 'hello'"
        elif self.expertise == "通用":
             return f"智能体 '{self.name}': 通用处理结果"
        else:
            return f"智能体 '{self.name}': 无法处理该输入"

# 模拟智能体路由
class AgentRouter:
    def __init__(self, name):
        self.name = name

    def route(self, input_text, available_agents):
        """模拟智能体路由，根据输入选择合适的智能体处理"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 智能体路由 '{self.name}' 路由输入：'{input_text}'")
        time.sleep(random.uniform(0.2, 0.5))
        # 这里使用简单的规则路由，实际应用中，可以使用更复杂的模型进行路由
        if "天气" in input_text:
            return [agent for agent in available_agents if agent.expertise == "天气"]
        elif "计算" in input_text or "+" in input_text or "-" in input_text or "*" in input_text or "/" in input_text:
            return [agent for agent in available_agents if agent.expertise == "计算"]
        elif "翻译" in input_text:
            return [agent for agent in available_agents if agent.expertise == "翻译"]
        else:
            return [agent for agent in available_agents if agent.expertise == "通用"]

# 模拟共识层集成
class ConsensusLayer:
    def __init__(self, name):
        self.name = name

    def aggregate(self, agent_outputs):
        """模拟共识层集成，将多个智能体的输出进行整合"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 共识层 '{self.name}' 集成输出：'{agent_outputs}'")
        time.sleep(random.uniform(0.3, 0.7))
        # 这里使用简单的规则进行整合，实际应用中，可以使用更复杂的模型进行整合
        if not agent_outputs:
           return "没有智能体的输出"
        elif len(agent_outputs) == 1:
            return agent_outputs[0]
        else:
            return f"共识层集成输出：{', '.join([f'{idx+1}: {output}' for idx, output in enumerate(agent_outputs)])}"


# 模拟反馈机制
class FeedbackMechanism:
  def __init__(self, name):
    self.name = name
    self.logs = []

  def record_feedback(self, final_output, input_text):
    """记录性能反馈"""
    feedback_entry = {
        "timestamp": datetime.datetime.now(),
        "input_text": input_text,
        "output": final_output,
       }
    self.logs.append(feedback_entry)
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 反馈机制 '{self.name}' 记录性能反馈：{feedback_entry}")


  def adjust_strategy(self):
    """根据反馈调整共识策略（简单模拟）"""
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 反馈机制 '{self.name}' 根据反馈调整共识策略")
    time.sleep(random.uniform(0.5, 1.0))
    if self.logs and any ("错误" in entry["output"] for entry in self.logs): # 如果有输出错误，就简单打印一条信息，实际可以根据反馈修改共识策略
       print (f"{datetime.datetime.now().strftime('%H:%M:%S')}: 反馈机制 '{self.name}' 发现有错误，建议调整共识策略")
       return True
    return False



# 初始化组件
agents = [
    IntelligentAgent("天气智能体", "天气"),
    IntelligentAgent("计算智能体", "计算"),
    IntelligentAgent("翻译智能体", "翻译"),
    IntelligentAgent("通用智能体", "通用")
]

router = AgentRouter("智能体路由")
consensus_layer = ConsensusLayer("共识层")
feedback = FeedbackMechanism("反馈机制")

# 主流程
def process_input(input_text):
    """处理输入的整体流程"""
    print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')}: 接收到输入：'{input_text}'")

    # 1. 智能体路由
    selected_agents = router.route(input_text, agents)

    # 2. 每个智能体处理输入
    agent_outputs = [agent.process_input(input_text if agent.expertise == "计算" else input_text) for agent in selected_agents]


    # 3. 共识层集成
    consensus_output = consensus_layer.aggregate(agent_outputs)

    # 4. 输出最终结果
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 最终输出: '{consensus_output}'")

    # 5. 反馈
    feedback.record_feedback(consensus_output, input_text)
    if feedback.adjust_strategy():
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}:  系统需要根据反馈调整策略，以便更好地处理后续输入\n")

    return consensus_output

# 示例输入
inputs = [
    "今天天气怎么样？",
    "计算 123 + 456",
    "翻译 你好",
    "我需要一个蛋糕",
    "今天天气如何？计算 5 * 3",
    "翻译 '再见'",
    "请问现在几点了？",
    "计算 1 / 0"  # 加入一个会计算错误的输入，来观察反馈机制
]

for input_text in inputs:
    process_input(input_text)
    time.sleep(1) # 加入时间间隔


06:09:14: 接收到输入：'今天天气怎么样？'
06:09:14: 智能体路由 '智能体路由' 路由输入：'今天天气怎么样？'
06:09:15: 智能体 '天气智能体' 处理输入：'今天天气怎么样？'
06:09:15: 共识层 '共识层' 集成输出：'["智能体 '天气智能体': 天气预报是晴天"]'
06:09:15: 最终输出: '智能体 '天气智能体': 天气预报是晴天'
06:09:15: 反馈机制 '反馈机制' 记录性能反馈：{'timestamp': datetime.datetime(2025, 1, 6, 6, 9, 15, 949235), 'input_text': '今天天气怎么样？', 'output': "智能体 '天气智能体': 天气预报是晴天"}
06:09:15: 反馈机制 '反馈机制' 根据反馈调整共识策略

06:09:17: 接收到输入：'计算 123 + 456'
06:09:17: 智能体路由 '智能体路由' 路由输入：'计算 123 + 456'
06:09:18: 智能体 '计算智能体' 处理输入：'计算 123 + 456'
06:09:18: 共识层 '共识层' 集成输出：'["智能体 '计算智能体': 计算错误"]'
06:09:19: 最终输出: '智能体 '计算智能体': 计算错误'
06:09:19: 反馈机制 '反馈机制' 记录性能反馈：{'timestamp': datetime.datetime(2025, 1, 6, 6, 9, 19, 33262), 'input_text': '计算 123 + 456', 'output': "智能体 '计算智能体': 计算错误"}
06:09:19: 反馈机制 '反馈机制' 根据反馈调整共识策略
06:09:19: 反馈机制 '反馈机制' 发现有错误，建议调整共识策略
06:09:19:  系统需要根据反馈调整策略，以便更好地处理后续输入


06:09:20: 接收到输入：'翻译 你好'
06:09:20: 智能体路由 '智能体路由' 路由输入：'翻译 你好'
06:09:21: 智能体 '翻译智能体' 处理输入：'翻译 你好'
06:09:21: 共识层 '共识层' 集成输出：'["智能体 '翻译智能体': 翻译结果：'你好' is 'hell

In [ ]:
# -*- coding: utf-8 -*-
"""
# 智能体系统流程演示

本 Jupyter Notebook 演示了一个多智能体协同处理数据并进行反馈优化的系统流程。
该流程对应了图片中的结构，模拟了数据输入、智能体路由、智能体处理、共识层集成、
反馈机制等环节。

"""

import numpy as np
import random
from typing import List, Callable

# --- 1. 数据输入 ---
def generate_input_data():
    """
    生成模拟的输入数据，这里使用简单的随机数作为示例
    """
    return random.random() # 生成 0 到 1 之间的随机浮点数

# --- 2. 智能体定义 ---
class SmartAgent:
    def __init__(self, name: str, processing_function: Callable):
        """
        初始化智能体
        Args:
            name: 智能体名称
            processing_function: 处理输入数据的函数
        """
        self.name = name
        self.process = processing_function

    def process_data(self, data):
        """
        处理输入数据
        Args:
            data: 输入数据
        Returns:
            处理结果
        """
        return self.process(data)

# --- 3. 智能体路由 ---
def smart_agent_routing(data, agents: List[SmartAgent]):
    """
    智能体路由函数：这里使用简单的随机选择智能体的方法
    Args:
        data: 输入数据
        agents: 所有智能体的列表
    Returns:
        一个字典，键为智能体名称，值为处理结果
    """
    chosen_agents = random.sample(agents, random.randint(1, len(agents))) # 随机选择1到所有智能体
    results = {}
    for agent in chosen_agents:
        results[agent.name] = agent.process_data(data) # 调用智能体处理数据
    return results

# --- 4. 共识层集成 ---
def consensus_integration(agent_results: dict, strategy:str = 'average'):
    """
    共识层集成函数，这里使用平均值或加权平均数作为示例
    Args:
        agent_results: 各个智能体的处理结果字典
        strategy: 集成策略，默认为'average'，可选'weighted_average'
    Returns:
        共识结果
    """

    if strategy == 'average':
        results = list(agent_results.values())
        if results:
            return sum(results) / len(results)
        else:
            return 0

    elif strategy == 'weighted_average':
        weights = np.random.rand(len(agent_results))
        values = np.array(list(agent_results.values()))
        return np.dot(weights / weights.sum(), values)

    else:
        raise ValueError("Unsupported integration strategy")

# --- 5. 反馈机制 ---
def feedback_mechanism(output, expected_output):
    """
    反馈机制，计算输出和期望输出之间的误差
    Args:
        output: 系统输出
        expected_output: 期望输出
    Returns:
        误差值
    """
    return abs(output - expected_output)

def adjust_integration_strategy(error, current_strategy):
    """
    调整共识集成策略, 这里示例使用简单的规则
    Args:
        error: 系统误差
        current_strategy: 当前策略
    Returns:
        调整后的策略
    """
    if error > 0.2:
        if current_strategy == 'average':
           return 'weighted_average'
    return current_strategy #如果误差不大则保持当前策略

# --- 6. 模拟系统运行 ---
def run_system(num_iterations: int = 10, agents: List[SmartAgent] = None, initial_strategy='average'):
    """
    模拟系统运行
    Args:
        num_iterations: 迭代次数
        agents: 所有智能体的列表
        initial_strategy: 初始共识集成策略
    """
    if agents is None:
        agents = create_default_agents()  # 如果没有传递，则创建默认智能体
    strategy = initial_strategy
    errors = [] # 用于记录每次迭代的误差
    for i in range(num_iterations):
        input_data = generate_input_data()
        print(f"\nIteration {i+1}: Input Data = {input_data:.2f}")
        agent_results = smart_agent_routing(input_data, agents) # 智能体路由
        print("  Agent Results:", agent_results)
        output = consensus_integration(agent_results, strategy)  # 共识集成
        print(f"  Consensus Output: {output:.2f}")
        expected_output = input_data * 2  # 示例，期望输出是输入的两倍
        error = feedback_mechanism(output, expected_output)  # 反馈机制
        print(f"  Error: {error:.2f}")
        errors.append(error)
        strategy = adjust_integration_strategy(error, strategy) # 调整策略
        print(f"  Updated Strategy: {strategy}")

    print('\nErrors per iteration:', errors)
    print(f"Final strategy: {strategy}")

# --- 7. 定义默认智能体 ---
def create_default_agents():
    """
     创建一组默认智能体
    Returns:
        智能体列表
    """
    agents = [
        SmartAgent("Agent 1", lambda x: x * 1.1),
        SmartAgent("Agent 2", lambda x: x * 0.9),
        SmartAgent("Agent 3", lambda x: x * 1.2),
        SmartAgent("Agent 4", lambda x: x * 0.8),
    ]
    return agents


# --- 主函数 ---
if __name__ == '__main__':
    print("# Start Simulation #\n")
    run_system(num_iterations = 10) # 运行模拟系统

# Start Simulation #


Iteration 1: Input Data = 0.64
  Agent Results: {'Agent 1': 0.7081425234912809, 'Agent 4': 0.5150127443572953, 'Agent 2': 0.5793893374019571, 'Agent 3': 0.7725191165359427}
  Consensus Output: 0.64
  Error: 0.64
  Updated Strategy: weighted_average

Iteration 2: Input Data = 0.47
  Agent Results: {'Agent 4': 0.37839675794540073, 'Agent 3': 0.567595136918101, 'Agent 1': 0.520295542174926}
  Consensus Output: 0.51
  Error: 0.43
  Updated Strategy: weighted_average

Iteration 3: Input Data = 0.21
  Agent Results: {'Agent 2': 0.18924620598973976}
  Consensus Output: 0.19
  Error: 0.23
  Updated Strategy: weighted_average

Iteration 4: Input Data = 0.41
  Agent Results: {'Agent 2': 0.3720555525875513}
  Consensus Output: 0.37
  Error: 0.45
  Updated Strategy: weighted_average

Iteration 5: Input Data = 0.85
  Agent Results: {'Agent 1': 0.9367585309024407}
  Consensus Output: 0.94
  Error: 0.77
  Updated Strategy: weighted_average

Iteration 6: Input Data = 0.05
  Agen

### 多任务微调 Fine-Tuning LLM's For Multiple Tasks

![图片描述](https://drive.google.com/uc?id=1zCLbJZdu0oUcXvFMaPptPHPoD5VbdtyP)

In [11]:
# 导入必要的库
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# 定义多任务模型类
class MultiTaskModel:
    def __init__(self):
        # 初始化基座大语言模型
        self.base_model_name = "t5-small"
        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.base_model_name)
        self.pipeline = pipeline('text2text-generation', model=self.model, tokenizer=self.tokenizer)

    def process_input(self, input_text):
        # 处理输入并生成输出
        output = self.pipeline(input_text, max_length=50, num_return_sequences=1)
        return output[0]['generated_text']

# 主函数
def main():
    # 创建多任务模型实例
    multi_task_model = MultiTaskModel()

    # 获取用户输入
    user_input = input("请输入文本: ")

    # 多任务模型处理
    shared_representation = multi_task_model.process_input(user_input)
    print("共享表示:", shared_representation)

    # 特定任务分发
    task_a_output = multi_task_model.process_input(f"任务A: {shared_representation}")
    task_b_output = multi_task_model.process_input(f"任务B: {shared_representation}")
    task_c_output = multi_task_model.process_input(f"任务C: {shared_representation}")

    # 输出结果
    print("任务A输出:", task_a_output)
    print("任务B输出:", task_b_output)
    print("任务C输出:", task_c_output)

if __name__ == "__main__":
    main()

Device set to use cpu


请输入文本: teste
共享表示: test test
任务A输出: Test test
任务B输出: Test test
任务C输出: Test test


In [10]:
import time
import datetime
import random

# 模拟基座大语言模型
class BaseLanguageModel:
    def __init__(self, name):
        self.name = name

    def fine_tune(self, task_data):
       """模拟基座大模型微调"""
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 基座大模型 '{self.name}' 根据任务数据进行微调：'{task_data}'")
       time.sleep(random.uniform(0.5, 1.5))
       return f"微调后的模型，可以处理不同的任务"


# 模拟多任务模型
class MultiTaskModel:
    def __init__(self, name, base_model):
        self.name = name
        self.base_model = base_model
        self.shared_representation = None # 共享表示层

    def process_input(self, task_data):
       """模拟多任务模型处理不同任务的输入，生成共享表示"""
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 多任务模型 '{self.name}' 处理输入数据：'{task_data}'")
       time.sleep(random.uniform(0.3, 0.8))
       # 模拟生成共享表示层
       self.shared_representation = f"共享表示层：处理后的数据：'{task_data}'"
       return self.shared_representation


# 模拟共享层
class SharedLayer:
    def __init__(self, name):
       self.name = name

    def get_shared_representation(self, shared_representation):
        """模拟共享层，接收多任务模型的输出，生成特定任务分发需要的表示"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 共享层 '{self.name}' 获取共享表示：'{shared_representation}'")
        time.sleep(random.uniform(0.2, 0.5))
        return f"特定任务分发表示： '{shared_representation}'"

# 模拟特定任务分发
class TaskDispatcher:
    def __init__(self, name):
        self.name = name

    def dispatch(self, shared_representation):
        """模拟任务分发，根据共享表示，将输入分配到不同的任务处理模块"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 任务分发器 '{self.name}' 分发任务，共享表示：'{shared_representation}'")
        time.sleep(random.uniform(0.2, 0.5))
        if "任务A" in shared_representation:
            return {"task": "任务A", "data": f"任务A的数据，基于 '{shared_representation}'"}
        elif "任务B" in shared_representation:
            return {"task": "任务B", "data": f"任务B的数据，基于 '{shared_representation}'"}
        elif "任务C" in shared_representation:
            return {"task": "任务C", "data": f"任务C的数据，基于 '{shared_representation}'"}
        else:
            return {"task":"未知", "data": f"未知任务，基于 '{shared_representation}'"}



# 模拟特定任务处理模块
class SpecificTaskProcessor:
    def __init__(self, name, task_name):
        self.name = name
        self.task_name = task_name

    def process_task(self, task_data):
      """模拟特定任务处理模块，处理特定任务数据，生成任务输出"""
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 特定任务处理器 '{self.name}' 处理任务 '{self.task_name}'，数据：'{task_data}'")
      time.sleep(random.uniform(0.3, 0.7))
      if self.task_name == "任务A":
         return "输出 A： 这是任务A的处理结果"
      elif self.task_name == "任务B":
          return "输出 B： 这是任务B的处理结果"
      elif self.task_name == "任务C":
          return "输出 C： 这是任务C的处理结果"
      else:
          return "未知输出:  无法处理该任务"


# 初始化组件
base_model = BaseLanguageModel("基座大语言模型")
multitask_model = MultiTaskModel("多任务模型", base_model)
shared_layer = SharedLayer("共享层")
task_dispatcher = TaskDispatcher("任务分发器")

task_processors = {
    "任务A": SpecificTaskProcessor("任务A处理器", "任务A"),
    "任务B": SpecificTaskProcessor("任务B处理器", "任务B"),
    "任务C": SpecificTaskProcessor("任务C处理器", "任务C"),
}

# 主流程
def process_input(inputs):
  """处理输入的整体流程"""
  print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')}: 接收到输入：'{inputs}'")

  # 1.  微调基座大语言模型
  fine_tuned_model = base_model.fine_tune(inputs)

  # 2.  多任务模型处理输入，生成共享表示层
  shared_representation = multitask_model.process_input(inputs)

  # 3.  获取共享表示
  task_representation = shared_layer.get_shared_representation(shared_representation)

  # 4.  特定任务分发
  dispatched_task = task_dispatcher.dispatch(task_representation)

  # 5.  特定任务处理
  if dispatched_task["task"] in task_processors:
     processor = task_processors[dispatched_task["task"]]
     task_output = processor.process_task(dispatched_task["data"])
  else:
    task_output = SpecificTaskProcessor("未知任务处理器","未知").process_task(dispatched_task["data"])


  # 6.  输出最终结果
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 最终输出：'{task_output}'\n")
  return task_output


# 示例输入
inputs_list = [
   "任务A 数据: 这是一个任务A的数据",
    "任务B 数据: 这是一个任务B的数据",
    "任务C 数据: 这是一个任务C的数据",
     "未知任务：这是一个未知任务的数据"
]


for input_data in inputs_list:
  process_input(input_data)
  time.sleep(1) # 加入时间间隔


06:11:47: 接收到输入：'任务A 数据: 这是一个任务A的数据'
06:11:47: 基座大模型 '基座大语言模型' 根据任务数据进行微调：'任务A 数据: 这是一个任务A的数据'
06:11:48: 多任务模型 '多任务模型' 处理输入数据：'任务A 数据: 这是一个任务A的数据'
06:11:48: 共享层 '共享层' 获取共享表示：'共享表示层：处理后的数据：'任务A 数据: 这是一个任务A的数据''
06:11:49: 任务分发器 '任务分发器' 分发任务，共享表示：'特定任务分发表示： '共享表示层：处理后的数据：'任务A 数据: 这是一个任务A的数据'''
06:11:49: 特定任务处理器 '任务A处理器' 处理任务 '任务A'，数据：'任务A的数据，基于 '特定任务分发表示： '共享表示层：处理后的数据：'任务A 数据: 这是一个任务A的数据''''
06:11:50: 最终输出：'输出 A： 这是任务A的处理结果'


06:11:51: 接收到输入：'任务B 数据: 这是一个任务B的数据'
06:11:51: 基座大模型 '基座大语言模型' 根据任务数据进行微调：'任务B 数据: 这是一个任务B的数据'
06:11:52: 多任务模型 '多任务模型' 处理输入数据：'任务B 数据: 这是一个任务B的数据'
06:11:52: 共享层 '共享层' 获取共享表示：'共享表示层：处理后的数据：'任务B 数据: 这是一个任务B的数据''
06:11:53: 任务分发器 '任务分发器' 分发任务，共享表示：'特定任务分发表示： '共享表示层：处理后的数据：'任务B 数据: 这是一个任务B的数据'''
06:11:53: 特定任务处理器 '任务B处理器' 处理任务 '任务B'，数据：'任务B的数据，基于 '特定任务分发表示： '共享表示层：处理后的数据：'任务B 数据: 这是一个任务B的数据''''
06:11:54: 最终输出：'输出 B： 这是任务B的处理结果'


06:11:55: 接收到输入：'任务C 数据: 这是一个任务C的数据'
06:11:55: 基座大模型 '基座大语言模型' 根据任务数据进行微调：'任务C 数据: 这是一个任务C的数据'
06:11:56: 多任务模型 '多任务模型' 处理输入数据：'任务C 数据: 这是一个任务

### 模块化组合 Modular Monolith LLM Approach With Composability

![图片描述](https://drive.google.com/uc?id=1uTF3Lv457E4-hkeIBEdUvsrl7gCqA46Q)

In [12]:
# 导入必要的库
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# 定义智能体类
class Agent:
    def __init__(self, name):
        self.name = name
        self.model_name = f"agent-{name}-model"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        self.pipeline = pipeline('text2text-generation', model=self.model, tokenizer=self.tokenizer)

    def process_input(self, input_text):
        # 处理输入并生成输出
        output = self.pipeline(input_text, max_length=50, num_return_sequences=1)
        return output[0]['generated_text']

# 主函数
def main():
    # 获取用户输入
    user_input = input("请输入文本: ")

    # 智能体选择：选择合适的智能体
    agents = [Agent(f"独立功能智能体 {i}") for i in range(1, 4)]  # 创建多个智能体实例

    # 输出集成：收集所有智能体的输出
    agent_outputs = []
    for agent in agents:
        output = agent.process_input(user_input)
        print(f"{agent.name} 的输出: {output}")
        agent_outputs.append(output)

    # 统一输出：整合所有智能体的输出
    unified_output = "\n".join(agent_outputs)
    print("统一输出:", unified_output)

    # 反馈机制：性能反馈和调整改进选择策略
    feedback = input("请输入反馈: ")
    print("反馈:", feedback)

if __name__ == "__main__":
    main()

请输入文本: 123123


OSError: Incorrect path_or_model_id: 'agent-独立功能智能体 1-model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [13]:
import time
import datetime
import random

# 模拟独立功能的智能体
class IndependentAgent:
    def __init__(self, name, expertise):
        self.name = name
        self.expertise = expertise

    def process_input(self, input_text):
        """模拟独立智能体处理输入，根据专业领域返回不同的输出"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 独立智能体 '{self.name}' 处理输入：'{input_text}'")
        time.sleep(random.uniform(0.3, 0.8))
        if self.expertise == "天气":
            return f"独立智能体 '{self.name}': 天气预报是晴天"
        elif self.expertise == "计算":
            try:
               return f"独立智能体 '{self.name}': 计算结果是：{eval(input_text)}"
            except:
                return f"独立智能体 '{self.name}': 计算错误"
        elif self.expertise == "翻译":
           return f"独立智能体 '{self.name}': 翻译结果是: '你好' is 'hello'"
        else:
            return f"独立智能体 '{self.name}': 无法处理该输入"


# 模拟智能体选择器
class AgentSelector:
    def __init__(self, name):
        self.name = name
        self.selection_strategy = "simple" # 默认选择策略

    def select_agents(self, input_text, available_agents):
        """模拟智能体选择，根据输入选择合适的智能体处理"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 智能体选择器 '{self.name}' 根据输入选择智能体：'{input_text}'")
        time.sleep(random.uniform(0.2, 0.5))
        # 使用简单选择策略
        if self.selection_strategy == "simple":
            if "天气" in input_text:
                return [agent for agent in available_agents if agent.expertise == "天气"]
            elif "计算" in input_text or "+" in input_text or "-" in input_text or "*" in input_text or "/" in input_text:
                return [agent for agent in available_agents if agent.expertise == "计算"]
            elif "翻译" in input_text:
                return [agent for agent in available_agents if agent.expertise == "翻译"]
            else:
                return [random.choice(available_agents)] # 随机选择一个智能体，如果不能匹配
        else: #  这里可以添加更多复杂的选择策略
            return [random.choice(available_agents)]

    def adjust_selection_strategy(self, feedback_result):
      """根据反馈结果调整选择策略（简单模拟）"""
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 智能体选择器 '{self.name}' 根据反馈调整选择策略")
      time.sleep(random.uniform(0.2,0.5))
      if "错误" in feedback_result:
         print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 智能体选择器 '{self.name}': 发现错误，将选择策略设置为更加复杂的选择策略(实际中可以使用更复杂的策略)")
         self.selection_strategy = "complex" # 这里只是简单地修改一个参数，实际可以使用更复杂的策略
      else:
         print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 智能体选择器 '{self.name}': 没有发现错误，维持当前的选择策略")

# 模拟输出集成器
class OutputAggregator:
    def __init__(self, name):
        self.name = name

    def aggregate(self, agent_outputs):
        """模拟输出集成，将多个智能体的输出进行整合"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 输出集成器 '{self.name}' 集成输出：'{agent_outputs}'")
        time.sleep(random.uniform(0.3, 0.7))
        if not agent_outputs:
           return "没有智能体的输出"
        elif len(agent_outputs) == 1:
            return agent_outputs[0]
        else:
            return f"集成输出：{', '.join([f'{idx+1}: {output}' for idx, output in enumerate(agent_outputs)])}"

# 模拟统一输出
class UnifiedOutput:
  def __init__(self, name):
    self.name = name

  def generate_unified_output(self, aggregated_output):
    """模拟生成统一的输出"""
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 统一输出器 '{self.name}' 生成统一的输出，输入：'{aggregated_output}'")
    time.sleep(random.uniform(0.2,0.5))
    return f"最终统一输出：'{aggregated_output}'"

# 模拟反馈机制
class FeedbackMechanism:
  def __init__(self, name):
    self.name = name
    self.logs = []

  def record_feedback(self, final_output, input_text):
      """记录性能反馈"""
      feedback_entry = {
          "timestamp": datetime.datetime.now(),
          "input_text": input_text,
          "output": final_output,
      }
      self.logs.append(feedback_entry)
      print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 反馈机制 '{self.name}' 记录反馈：{feedback_entry}")

  def get_feedback_result(self):
     """简单根据是否有输出错误来返回反馈结果"""
     if self.logs and any("错误" in entry["output"] for entry in self.logs):
          return "输出包含错误"
     else:
          return "输出没有错误"



# 初始化组件
agents = [
    IndependentAgent("天气智能体", "天气"),
    IndependentAgent("计算智能体", "计算"),
    IndependentAgent("翻译智能体", "翻译"),
    IndependentAgent("通用智能体","通用")
]

selector = AgentSelector("智能体选择器")
aggregator = OutputAggregator("输出集成器")
unified_output = UnifiedOutput("统一输出器")
feedback = FeedbackMechanism("反馈机制")

# 主流程
def process_input(input_text):
  """处理输入的整体流程"""
  print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')}: 接收到输入：'{input_text}'")

  # 1. 智能体选择
  selected_agents = selector.select_agents(input_text, agents)

  # 2. 每个智能体处理输入
  agent_outputs = [agent.process_input(input_text if agent.expertise == "计算" else input_text) for agent in selected_agents]

  # 3. 输出集成
  aggregated_output = aggregator.aggregate(agent_outputs)

  # 4. 统一输出
  final_output = unified_output.generate_unified_output(aggregated_output)

  # 5. 输出最终结果
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 最终输出：'{final_output}'")

  # 6. 反馈
  feedback.record_feedback(final_output, input_text)
  feedback_result = feedback.get_feedback_result()
  selector.adjust_selection_strategy(feedback_result)
  return final_output

# 示例输入
inputs = [
    "今天天气怎么样？",
    "计算 123 + 456",
    "翻译 你好",
    "我需要一个蛋糕",
    "今天天气如何？计算 5 * 3",
    "翻译 '再见'",
    "请问现在几点了？",
    "计算 1 / 0" # 加入一个会计算错误的输入来触发选择策略调整
]


for input_text in inputs:
    process_input(input_text)
    time.sleep(1) # 加入时间间隔


06:13:56: 接收到输入：'今天天气怎么样？'
06:13:56: 智能体选择器 '智能体选择器' 根据输入选择智能体：'今天天气怎么样？'
06:13:56: 独立智能体 '天气智能体' 处理输入：'今天天气怎么样？'
06:13:56: 输出集成器 '输出集成器' 集成输出：'["独立智能体 '天气智能体': 天气预报是晴天"]'
06:13:57: 统一输出器 '统一输出器' 生成统一的输出，输入：'独立智能体 '天气智能体': 天气预报是晴天'
06:13:57: 最终输出：'最终统一输出：'独立智能体 '天气智能体': 天气预报是晴天''
06:13:57: 反馈机制 '反馈机制' 记录反馈：{'timestamp': datetime.datetime(2025, 1, 6, 6, 13, 57, 712626), 'input_text': '今天天气怎么样？', 'output': "最终统一输出：'独立智能体 '天气智能体': 天气预报是晴天'"}
06:13:57: 智能体选择器 '智能体选择器' 根据反馈调整选择策略
06:13:57: 智能体选择器 '智能体选择器': 发现错误，将选择策略设置为更加复杂的选择策略(实际中可以使用更复杂的策略)

06:13:58: 接收到输入：'计算 123 + 456'
06:13:58: 智能体选择器 '智能体选择器' 根据输入选择智能体：'计算 123 + 456'
06:13:59: 独立智能体 '翻译智能体' 处理输入：'计算 123 + 456'
06:13:59: 输出集成器 '输出集成器' 集成输出：'["独立智能体 '翻译智能体': 翻译结果是: '你好' is 'hello'"]'
06:14:00: 统一输出器 '统一输出器' 生成统一的输出，输入：'独立智能体 '翻译智能体': 翻译结果是: '你好' is 'hello''
06:14:00: 最终输出：'最终统一输出：'独立智能体 '翻译智能体': 翻译结果是: '你好' is 'hello'''
06:14:00: 反馈机制 '反馈机制' 记录反馈：{'timestamp': datetime.datetime(2025, 1, 6, 6, 14, 0, 480628), 'input_text': '计算 123 + 4

### 知识库驱动 Utilizing Knowledge Graphs with LLM's

![图片描述](https://drive.google.com/uc?id=1OSf7xfeOCSA4DvbnrmgYQDa6ihrgNBE3)

In [15]:
# 导入必要的库
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import requests

# 定义大语言模型类
class LargeLanguageModel:
    def __init__(self):
        self.model_name = "t5-small"  # 使用T5模型作为示例
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        self.pipeline = pipeline('text2text-generation', model=self.model, tokenizer=self.tokenizer)

    def process_input(self, input_text):
        # 处理输入并生成输出
        output = self.pipeline(input_text, max_length=50, num_return_sequences=1)
        return output[0]['generated_text']

# 定义知识图谱类
class KnowledgeGraph:
    def __init__(self):
        self.base_url = "http://example.com/knowledge-graph"

    def query(self, query_text):
        # 查询知识图谱
        response = requests.get(f"{self.base_url}/query", params={"query": query_text})
        return response.json()

    def update(self, update_text):
        # 更新知识图谱
        response = requests.post(f"{self.base_url}/update", json={"update": update_text})
        return response.json()

# 主函数
def main():
    # 创建大语言模型实例
    llm = LargeLanguageModel()

    # 获取用户输入
    user_input = input("请输入文本: ")

    # 大语言模型处理
    initial_output = llm.process_input(user_input)
    print("初始输出:", initial_output)

    # 知识图谱查询
    kg = KnowledgeGraph()
    query_result = kg.query(initial_output)
    print("知识图谱查询结果:", query_result)

    # 更新知识图谱
    update_result = kg.update(query_result)
    print("知识图谱更新结果:", update_result)

    # 再次使用大语言模型处理
    final_output = llm.process_input(update_result)
    print("最终输出:", final_output)

if __name__ == "__main__":
    main()

Device set to use cpu


请输入文本: testet
初始输出: testet


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
import time
import datetime
import random

# 模拟大语言模型
class LargeLanguageModel:
    def __init__(self, name):
        self.name = name

    def generate_response(self, input_text, knowledge_graph=None):
        """模拟大语言模型生成回复，包含对知识图谱的查询"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型 '{self.name}' 处理输入：'{input_text}'")
        time.sleep(random.uniform(0.5, 1.5))
        if knowledge_graph:
          print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型使用知识图谱: '{knowledge_graph.query(input_text)}'")
          return f"大语言模型 '{self.name}' 基于知识图谱和输入 '{input_text}' 生成的回复"
        else:
           return f"大语言模型 '{self.name}'  基于输入 '{input_text}' 生成的回复"

    def generate_knowledge_graph_query(self, input_text):
       """模拟大语言模型生成知识图谱查询"""
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型 '{self.name}' 为知识图谱生成查询语句: '{input_text}'")
       time.sleep(random.uniform(0.3, 0.8))
       return f"查询：与'{input_text}' 相关的信息"

    def generate_ontology_from_text(self, text):
        """模拟大语言模型从文本生成本体"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型 '{self.name}' 从文本生成本体：'{text}'")
        time.sleep(random.uniform(0.5, 1.5))
        return f"从文本 '{text}' 生成的本体"

    def generate_prompt_from_ontology(self, ontology):
       """模拟大语言模型从本体生成文本提示词"""
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 大语言模型 '{self.name}' 从本体生成文本提示词：'{ontology}'")
       time.sleep(random.uniform(0.3, 0.8))
       return f"从本体 '{ontology}' 生成的文本提示词"


# 模拟知识图谱
class KnowledgeGraph:
    def __init__(self, name):
      self.name = name
      self.knowledge = {}  # 使用字典存储知识图谱，键值对

    def query(self, query_text):
        """模拟知识图谱查询"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 知识图谱 '{self.name}' 查询信息: '{query_text}'")
        if self.knowledge:
           for key, value in self.knowledge.items():
                if query_text in key:
                    return value
        return f"知识图谱 '{self.name}' 没有找到与 '{query_text}' 相关的信息"


    def update(self, entity, value):
        """模拟更新知识图谱，简单用键值对存储"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 知识图谱 '{self.name}' 更新信息：实体 '{entity}'， 值为'{value}'")
        self.knowledge[entity] = value


# 模拟知识数据源
class KnowledgeDataSource:
  def __init__(self, name):
    self.name = name
    self.data = {}

  def supply_data(self, query_text):
    """模拟知识数据源根据查询提供数据"""
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 知识数据源 '{self.name}' 根据查询 '{query_text}' 提供数据")
    time.sleep(random.uniform(0.3, 0.8))
    if self.data and query_text in self.data:
        return self.data[query_text]
    return f"知识数据源 '{self.name}': 没有找到与 '{query_text}' 相关的数据"

  def update_data(self, entity, value):
    """模拟知识数据源更新数据"""
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 知识数据源 '{self.name}' 更新数据: 实体 '{entity}', 值为 '{value}'")
    self.data[entity] = value



# 初始化组件
llm = LargeLanguageModel("大语言模型")
knowledge_graph = KnowledgeGraph("知识图谱")
knowledge_source = KnowledgeDataSource("知识数据源")

# 主流程
def process_input(input_text):
  """处理输入的整体流程"""
  print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')}: 接收到输入：'{input_text}'")

  # 1. 大语言模型生成回复 (可能需要使用知识图谱)
  query_for_kg = llm.generate_knowledge_graph_query(input_text)
  response_from_llm = llm.generate_response(input_text, knowledge_graph)

  # 2.  知识图谱查询，用于大模型生成信息
  kg_result = knowledge_graph.query(query_for_kg)

  # 3.  使用大语言模型从知识数据源生成本体, 并更新知识图谱
  data_from_source = knowledge_source.supply_data(query_for_kg)
  ontology_from_text = llm.generate_ontology_from_text(data_from_source)

  # 4.  更新知识图谱
  knowledge_graph.update(query_for_kg, ontology_from_text)

  # 5. 使用大语言模型从知识图谱生成文本提示词
  prompt_from_ontology = llm.generate_prompt_from_ontology(ontology_from_text)
  # 6. 更新知识数据源
  knowledge_source.update_data(query_for_kg, prompt_from_ontology)


  # 7. 输出最终结果
  print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 最终输出：'{response_from_llm}'\n")
  return response_from_llm

# 示例输入
inputs = [
    "今天天气怎么样？",
    "谁是爱因斯坦？",
     "介绍一下巴黎",
     "最新的新闻是什么？"
]

for input_text in inputs:
    process_input(input_text)
    time.sleep(1) # 加入时间间隔


06:15:09: 接收到输入：'今天天气怎么样？'
06:15:09: 大语言模型 '大语言模型' 为知识图谱生成查询语句: '今天天气怎么样？'
06:15:09: 大语言模型 '大语言模型' 处理输入：'今天天气怎么样？'
06:15:10: 知识图谱 '知识图谱' 查询信息: '今天天气怎么样？'
06:15:10: 大语言模型使用知识图谱: '知识图谱 '知识图谱' 没有找到与 '今天天气怎么样？' 相关的信息'
06:15:10: 知识图谱 '知识图谱' 查询信息: '查询：与'今天天气怎么样？' 相关的信息'
06:15:10: 知识数据源 '知识数据源' 根据查询 '查询：与'今天天气怎么样？' 相关的信息' 提供数据
06:15:10: 大语言模型 '大语言模型' 从文本生成本体：'知识数据源 '知识数据源': 没有找到与 '查询：与'今天天气怎么样？' 相关的信息' 相关的数据'
06:15:11: 知识图谱 '知识图谱' 更新信息：实体 '查询：与'今天天气怎么样？' 相关的信息'， 值为'从文本 '知识数据源 '知识数据源': 没有找到与 '查询：与'今天天气怎么样？' 相关的信息' 相关的数据' 生成的本体'
06:15:11: 大语言模型 '大语言模型' 从本体生成文本提示词：'从文本 '知识数据源 '知识数据源': 没有找到与 '查询：与'今天天气怎么样？' 相关的信息' 相关的数据' 生成的本体'
06:15:12: 知识数据源 '知识数据源' 更新数据: 实体 '查询：与'今天天气怎么样？' 相关的信息', 值为 '从本体 '从文本 '知识数据源 '知识数据源': 没有找到与 '查询：与'今天天气怎么样？' 相关的信息' 相关的数据' 生成的本体' 生成的文本提示词'
06:15:12: 最终输出：'大语言模型 '大语言模型' 基于知识图谱和输入 '今天天气怎么样？' 生成的回复'


06:15:13: 接收到输入：'谁是爱因斯坦？'
06:15:13: 大语言模型 '大语言模型' 为知识图谱生成查询语句: '谁是爱因斯坦？'
06:15:13: 大语言模型 '大语言模型' 处理输入：'谁是爱因斯坦？'
06:15:14: 知识图谱 '知识图谱' 查询信息: '谁是爱因斯坦？'
06:15:14: 大语言模型使用知识图谱: '知

### 双模评估 Red & Blue Team Dual-Model Evaluation

![图片描述](https://drive.google.com/uc?id=1GutKqndPTRkp9MtLlTMofRA2At6FBM3-)

In [16]:
# 导入必要的库
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# 定义学生大模型类
class StudentModel:
    def __init__(self):
        self.model_name = "t5-small"  # 使用T5模型作为示例
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        self.pipeline = pipeline('text2text-generation', model=self.model, tokenizer=self.tokenizer)

    def process_input(self, input_text):
        # 处理输入并生成中间输出
        output = self.pipeline(input_text, max_length=50, num_return_sequences=1)
        return output[0]['generated_text']

# 定义老师大模型类
class TeacherModel:
    def __init__(self):
        self.model_name = "t5-large"  # 使用更大的T5模型作为示例
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        self.pipeline = pipeline('text2text-generation', model=self.model, tokenizer=self.tokenizer)

    def evaluate_output(self, input_text):
        # 评估中间输出并生成最终提示词
        output = self.pipeline(input_text, max_length=50, num_return_sequences=1)
        return output[0]['generated_text']

# 主函数
def main():
    # 创建学生大模型实例
    student_model = StudentModel()

    # 获取用户输入
    user_input = input("请输入文本: ")

    # 学生大模型处理
    intermediate_output = student_model.process_input(user_input)
    print("中间输出:", intermediate_output)

    # 创建老师大模型实例
    teacher_model = TeacherModel()

    # 老师大模型评估
    evaluation_result = teacher_model.evaluate_output(intermediate_output)
    print("评估结果:", evaluation_result)

    # 根据评估结果调整和改进模型
    adjusted_output = student_model.process_input(evaluation_result)
    print("最终输出:", adjusted_output)

if __name__ == "__main__":
    main()

Device set to use cpu


请输入文本: 今天天气
中间输出: 


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


评估结果: 
最终输出: 


In [17]:
import time
import datetime
import random

# 模拟学生大模型
class StudentLargeModel:
    def __init__(self, name):
        self.name = name
        self.model_performance = 0.5  # 模拟模型表现，初始值为0.5
        self.model_strategy = "simple" # 模拟模型策略，初始值为 simple

    def complete_task(self, input_text):
        """模拟学生大模型完成任务，基于当前的模型策略"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 学生大模型 '{self.name}' 处理输入: '{input_text}'")
        time.sleep(random.uniform(0.5, 1.5))

        if self.model_strategy == "simple":
            # 模拟不同的策略，基于输入生成不同的输出
           if "数学" in input_text:
             try:
                result = eval(input_text.split("数学")[-1].strip())
                output = f"学生大模型 '{self.name}':  使用简单策略，计算结果为：{result}"
             except:
                output = f"学生大模型 '{self.name}': 使用简单策略，计算错误"
           elif "翻译" in input_text:
              output = f"学生大模型 '{self.name}':  使用简单策略，翻译结果为:  '你好' is 'hello'"
           else:
               output = f"学生大模型 '{self.name}':  使用简单策略，处理结果"
        else: # 使用复杂的模型策略
             output = f"学生大模型 '{self.name}':  使用复杂策略，处理结果"

        return output


    def adjust_model(self, feedback_prompt):
       """模拟学生模型调整自身，根据反馈提示词"""
       print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 学生大模型 '{self.name}' 根据提示词调整模型：'{feedback_prompt}'")
       time.sleep(random.uniform(0.3, 0.8))
       if "做得不好" in feedback_prompt:
            print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 学生大模型 '{self.name}' 根据提示词，将模型策略设置为复杂策略")
            self.model_strategy = "complex"
       else:
           print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 学生大模型 '{self.name}' 根据提示词，维持模型策略")
       self.model_performance += random.uniform(0.05, 0.15)  # 模拟模型性能提升



# 模拟老师大模型
class TeacherLargeModel:
    def __init__(self, name):
        self.name = name

    def evaluate_task(self, task_output):
        """模拟老师大模型评估任务，生成反馈提示词"""
        print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 老师大模型 '{self.name}' 评估任务输出：'{task_output}'")
        time.sleep(random.uniform(0.5, 1.5))
        # 这里使用简单规则评估任务输出
        if "计算错误" in task_output:
           return "老师大模型提示：你的计算结果做得不好，请重新检查"
        elif "你好" not in task_output and "翻译结果" in task_output:
             return "老师大模型提示：你的翻译做得不够准确，需要翻译'你好'"
        else:
            return "老师大模型提示：你的任务完成的很好"

# 初始化组件
student_model = StudentLargeModel("学生大模型")
teacher_model = TeacherLargeModel("老师大模型")

# 主流程
def process_input(input_text):
    """处理输入的整体流程"""
    print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')}: 接收到输入：'{input_text}'")

    # 1.  学生模型完成任务
    student_output = student_model.complete_task(input_text)

    # 2.  教师模型评估任务输出
    feedback_prompt = teacher_model.evaluate_task(student_output)


    # 3.  学生模型调整
    student_model.adjust_model(feedback_prompt)

    # 4.  输出最终结果
    print(f"{datetime.datetime.now().strftime('%H:%M:%S')}: 最终输出：'{student_output}'\n")
    return student_output

# 示例输入
inputs = [
    "数学计算 1 + 2",
    "翻译 你好",
    "数学计算 1 / 0",
    "请你总结一下今天的新闻"
]

for input_text in inputs:
    process_input(input_text)
    time.sleep(1) # 加入时间间隔


06:20:54: 接收到输入：'数学计算 1 + 2'
06:20:54: 学生大模型 '学生大模型' 处理输入: '数学计算 1 + 2'
06:20:55: 老师大模型 '老师大模型' 评估任务输出：'学生大模型 '学生大模型': 使用简单策略，计算错误'
06:20:57: 学生大模型 '学生大模型' 根据提示词调整模型：'老师大模型提示：你的计算结果做得不好，请重新检查'
06:20:57: 学生大模型 '学生大模型' 根据提示词，将模型策略设置为复杂策略
06:20:57: 最终输出：'学生大模型 '学生大模型': 使用简单策略，计算错误'


06:20:58: 接收到输入：'翻译 你好'
06:20:58: 学生大模型 '学生大模型' 处理输入: '翻译 你好'
06:20:59: 老师大模型 '老师大模型' 评估任务输出：'学生大模型 '学生大模型':  使用复杂策略，处理结果'
06:21:00: 学生大模型 '学生大模型' 根据提示词调整模型：'老师大模型提示：你的任务完成的很好'
06:21:00: 学生大模型 '学生大模型' 根据提示词，维持模型策略
06:21:00: 最终输出：'学生大模型 '学生大模型':  使用复杂策略，处理结果'


06:21:01: 接收到输入：'数学计算 1 / 0'
06:21:01: 学生大模型 '学生大模型' 处理输入: '数学计算 1 / 0'
06:21:02: 老师大模型 '老师大模型' 评估任务输出：'学生大模型 '学生大模型':  使用复杂策略，处理结果'
06:21:04: 学生大模型 '学生大模型' 根据提示词调整模型：'老师大模型提示：你的任务完成的很好'
06:21:04: 学生大模型 '学生大模型' 根据提示词，维持模型策略
06:21:04: 最终输出：'学生大模型 '学生大模型':  使用复杂策略，处理结果'


06:21:05: 接收到输入：'请你总结一下今天的新闻'
06:21:05: 学生大模型 '学生大模型' 处理输入: '请你总结一下今天的新闻'
06:21:06: 老师大模型 '老师大模型' 评估任务输出：'学生大模型 '学生大模型':  使用复杂策略，处理结果'
06:21:08: 学生大模型 '学生大模型' 根据提示词调整模型：'老师大模型提示：你的任务完成的